<a href="https://colab.research.google.com/github/jgvinholi/sar_atr_functions/blob/master/SAR_CDA_WLR_CNN_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Retrieve all needed files:

In [ ]:
# Mount GDrive folder:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Get files from Gdrive:
!cp -r '/content/drive/My Drive/SAR_CD/' '/content/SAR_CD' -v 

In [ ]:
# Get updated functions from github:
%cd /content/SAR_CD/
!git clone https://github.com/jgvinholi/sar_atr_functions.git

### Download and import packages:

In [ ]:
!pip install tikzplotlib

In [ ]:
!pip install -U --pre tensorflow-gpu --quiet

In [ ]:
%cd /content/SAR_CD
%tensorflow_version 2.x
%load_ext autoreload
%autoreload 2

class noiseStruct(object):
  def __init__(self, speckle_var, gaussian_var, aug_negat_prob, rot_angle):
    self.gaussian_mean = 0
    self.speckle_mean = 1
    self.speckle_var = speckle_var
    self.gaussian_var = gaussian_var
    self.aug_negat_prob = aug_negat_prob
    self.rot_angle = rot_angle

from matplotlib.image import imread
from matplotlib import rcParams
import matplotlib.pyplot as plt
import cv2
import numpy as np
import scipy
import os, glob
import tensorflow as tf
import pickle
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.layers import Input, Dense, Flatten, ZeroPadding2D, Conv2D, Activation, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, Nadam
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.cluster import DBSCAN
from sklearn import metrics
from scipy.cluster.hierarchy import fclusterdata
import numba
from numba import jit, njit
import warnings
from operator import itemgetter 
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from joblib import Parallel, delayed
import multiprocessing
import importlib

# Import custom functions:
import sys
sys.path.insert(1, '/content/SAR_CD/sar_atr_functions/')
from basefunctions import *
from focalloss import *
from twostepsdetect_functions import *
# from twostepsdetect_functions import noiseStruct

/content/SAR_CD
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Defining the predetection model:

In [ ]:

def create_conv_model():
  A = Input(shape = (3000, 2000, 1) )
  B = Conv2D(16, (5, 5), name = 'conv0_5x5', padding = 'same', activation = 'relu')(A)
  C = Conv2D(16, (1, 1), name = 'conv1_1x1', padding = 'same', activation = 'relu')(B)
  C = Dropout(0.3)(C)
  D = Conv2D(8, (3, 3), name = 'conv2_3x3', padding = 'same', activation = 'relu')(C)
  # Out = Conv2D(1, (1, 1), name = 'out_1x1', padding = 'same', activation = 'sigmoid', bias_initializer=initializers.Constant(-4.595119) )(D)
  Out = Conv2D(1, (1, 1), name = 'out_1x1', padding = 'same', activation = 'sigmoid', bias_initializer=tf.keras.initializers.Constant(-4.59511985013459) )(D)
  model = Model(inputs = A, outputs = Out)
  adam = Adam(lr = 5e-5)
  # sgd = SGD(lr = 0.3, decay = 2e-4, momentum=0.7)
  model.compile(optimizer = adam, loss=[binary_focal_loss(gamma = 2, alpha = 0.9999)])
  return model 


### Defining class prediction model:

In [ ]:
def create_classpred_model():
  A = Input(shape = (window_size, window_size, 1) )
  # B = BatchNormalization()(A)
  
  B = Conv2D(16, (3, 3), name = 'conv0_3x3', padding = 'same', activation = 'relu', kernel_initializer=tf.keras.initializers.he_normal(), bias_initializer=tf.keras.initializers.he_normal() )(A)
  B = BatchNormalization()(B)
  C = Conv2D(16, (3, 3), name = 'conv1_3x3', padding = 'same', activation = 'relu', kernel_initializer=tf.keras.initializers.he_normal(), bias_initializer=tf.keras.initializers.he_normal())(B)
  C = MaxPooling2D(pool_size = (2, 2), name = 'maxp_1_2x2_s2', strides = (2, 2), padding = "same" )(C) # Output = 17x17
  C = BatchNormalization()(C)
  

  D = Conv2D(32, (3, 3), name = 'conv2_3x3', padding = 'same', activation = 'relu', kernel_initializer=tf.keras.initializers.he_normal(), bias_initializer=tf.keras.initializers.he_normal())(C)
  E = MaxPooling2D(pool_size = (2, 2), name = 'maxp_2_2x2_s2', strides = (2, 2), padding = "same" )(D) # Output = 9x9
  E = BatchNormalization()(E)
  # E = Dropout(0.3)(E)
  
  F = Conv2D(64, (3, 3), name = 'conv3_3x3', padding = 'same', activation = 'relu', kernel_initializer=tf.keras.initializers.he_normal(), bias_initializer=tf.keras.initializers.he_normal())(E) 
  G = MaxPooling2D(pool_size = (2, 2), name = 'maxp_3_2x2_s2', strides = (2, 2), padding = "same" )(F) # Output = 5x5
  G = BatchNormalization()(G)
  # G = Dropout(0.3)(G)

  H = Conv2D(64, (3, 3), name = 'conv4_3x3', padding = 'valid', activation = 'relu', kernel_initializer=tf.keras.initializers.he_normal(), bias_initializer=tf.keras.initializers.he_normal())(G) # Output = 3x3
  I = AveragePooling2D(pool_size = (3, 3), name = 'avp_0_3x3_s1', strides = (1, 1), padding = "valid" )(H) # Output = 1x1
  I = Dropout(0.3)(I)
  # I = BatchNormalization()(I)
  

  Out = Conv2D(1, (1, 1), name = 'conv4_1x1', padding = 'valid', activation = 'sigmoid', kernel_initializer=tf.keras.initializers.he_normal(), bias_initializer=tf.keras.initializers.Constant(-4.59511985013459))(I)
  model = Model(inputs = A, outputs = Out)
  adam = Adam(lr = 2e-5)
  model.compile(optimizer = adam, loss=[binary_focal_loss(gamma = 2, alpha = 0.9)])
  return model



### Retrieve windows to train classification model:

In [ ]:
 X_full_pixval_class_window, Y_class_window, X_full_pixval_class_window_noaug, Y_class_window_noaug = load_multiple_classification_gt()

In [ ]:
with open(datab_imgs_path + 'classification_data/' + 'xy_classification_noaug.pkl' , 'wb') as f:  # Python 3: open(..., 'wb')
      pickle.dump([X_full_pixval_class_window_noaug, Y_class_window_noaug], f)
      print('saved')

saved


### Train the classification model:

In [ ]:
def train_classpred(X_full_pixval_class_window, Y_class_window, X_full_pixval_class_window_noaug, Y_class_window_noaug, img_names):
  n_split = int(len(img_names)/4)
  print( str(n_split) + "-fold.")
  model_classconv = [create_classpred_model() for i in range(n_split)]
  model_classconv[0].summary()
  scaler = StandardScaler()
  kf = KFold(n_splits = n_split)
  early_stopping = EarlyStopping(monitor='val_loss', patience = 20, restore_best_weights = True)
  X_full_pixval_class_window, Y_class_window = np.asarray(X_full_pixval_class_window), np.asarray(Y_class_window)
  X_full_pixval_class_window_noaug, Y_class_window_noaug = np.asarray(X_full_pixval_class_window_noaug), np.asarray(Y_class_window_noaug)
  try:
    for j, (train_index, validation_index) in enumerate( kf.split(np.arange(len(img_names) ) ) ):
      print(train_index, validation_index)
      x_train, x_validation = np.concatenate(X_full_pixval_class_window[train_index]), np.concatenate(X_full_pixval_class_window_noaug[validation_index])
      y_train, y_validation = np.concatenate(Y_class_window[train_index]), np.concatenate(Y_class_window_noaug[validation_index])
      print(x_train.shape, x_validation.shape)
      x_train, x_validation = np.reshape(x_train, (x_train.shape[0], window_size, window_size, 1) ), np.reshape(x_validation, (x_validation.shape[0], window_size, window_size, 1) )
      y_train, y_validation = np.reshape( y_train, (y_train.shape[0], 1, 1) ), np.reshape( y_validation, (y_validation.shape[0], 1, 1) )
      print(x_train.shape, y_train.shape)
      model_classconv[j].fit(x_train, y_train, epochs = 15, batch_size = 100, validation_data = (x_validation, y_validation)) 
    return model_classconv
  except KeyboardInterrupt:
    return model_classconv


In [ ]:
model_classconv = train_classpred(X_full_pixval_class_window, Y_class_window, X_full_pixval_class_window_noaug, Y_class_window_noaug, whole_set)

### Training the detection model:

In [ ]:
def train_conv_kf(img_names):
  n_split = int(len(img_names)/4)
  model_conv = [create_conv_model() for i in range(n_split)]
  model_conv[0].summary()
  scaler = StandardScaler()
  kf = KFold(n_splits = n_split)
  j = 0
  early_stopping = EarlyStopping(monitor='val_loss', patience = 10)
  try:
    for train_index, validation_index in kf.split(np.arange( len(img_names ) ) ):
      print('Train: ' + str( itemgetter(*train_index)(img_names) ) ) 
      print('Validation: ' + str( itemgetter(*validation_index)(img_names) ) )
      x_train, x_validation = Images_vector_norm[train_index, :, :, :], Images_vector_norm[validation_index, :, :, :]
      y_train, y_validation = Y_full[train_index, :, :, :], Y_full[validation_index, :, :, :]
      x_train, x_validation, y_train, y_validation = x_train.astype(np.float32), x_validation.astype(np.float32), y_train.astype(np.float32), y_validation.astype(np.float32)
      model_conv[j].fit(x_train, y_train, epochs = 1100,
                        batch_size = 3, validation_data = (x_validation, y_validation) )#, 
                        #callbacks = [early_stopping])  
      j += 1
        
    print("KFold distribution.")
    w = np.arange(0, n_split*4)
    for s1, s2 in kf.split(w):
        print(s1, s2)
    return model_conv

  except KeyboardInterrupt:
    return model_conv

In [ ]:
model_conv = train_conv_kf(whole_set)

### Model ROC

In [ ]:
# thresholds = np.arange(0.005, 0.15, 0.00125)
kfold = 1
thresholds = np.arange(0, 1.005, 0.005)
pred_thresholds = np.linspace(0.5, 0.95, 11)
print(thresholds, len(thresholds))
print(pred_thresholds)
mean_f1_scores, mean_precisions, mean_recalls, mean_fprs = roc_multiple_images(model_conv, model_classconv, whole_set[0:20], whole_set, thresholds, pred_thresholds, 1)


### Load/save predetection model:

In [ ]:
# save_modelconv(model_conv, "convarch3_lr5em5_1100epochs_focal_KF_6models", 1)

In [ ]:
model_conv = load_modelconv("convarch3_lr5em5_1100epochs_focal_KF_6models", 1)
# model_conv = load_modelconv("convarch3_lr5em5_1100epochs_focal_KF", 1)

### Load/save classification model:

In [ ]:
# save_modelconv(model_classconv, "classarch16_lr2em5_15epoch_focal_beta2alpha0p9_0p7noise_allcenteredtechnique_gaussiannoise_6models", 1)

In [ ]:
# model_classconv = load_modelconv("classarch16_lr2em5_15epoch_focal_beta2alpha0p9_0p7noise_allcenteredtechnique_gaussiannoise", 1)
model_classconv = load_modelconv("classarch16_lr2em5_15epoch_focal_beta2alpha0p9_0p7noise_allcenteredtechnique_gaussiannoise_6models", 1)


### Predict all images

In [ ]:
for h in np.arange(0.8+0.02, 0.9, 0.02):
  print(h)
  for i in np.arange(0, 21, 4):
    detected_targets, false_positives = predict_and_save(model_conv, model_classconv, whole_set[i:i+4], detect_thresh = 0.68, classif_thresh = [h], save=1)